## install mecab on mac
```
brew install mecab mecab-ipadic  
pip install mecab-python3
```

## install neologd
```
git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
cd mecab-ipadic-neologd
./bin/install-mecab-ipadic-neologd -n
```

## get Japanese articles
livedoor ニュースコーパス

```
wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
tar xvzf ldcc-20140209.tar.gz
```

In [1]:
from urllib import request 
import logging
from pathlib import Path
import numpy as np
import tqdm
import re
import MeCab
from gensim import corpora, models

In [2]:
mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")

In [3]:
[line.split("\t") for line in mecab.parse("認めたくないものだな。自分自身の若さ故の過ちというものを。").split("\n")]

[['認め', 'ミトメ', '認める', '動詞-自立', '一段', '連用形'],
 ['たく', 'タク', 'たい', '助動詞', '特殊・タイ', '連用テ接続'],
 ['ない', 'ナイ', 'ない', '助動詞', '特殊・ナイ', '基本形'],
 ['もの', 'モノ', 'もの', '名詞-非自立-一般', '', ''],
 ['だ', 'ダ', 'だ', '助動詞', '特殊・ダ', '基本形'],
 ['な', 'ナ', 'な', '助詞-終助詞', '', ''],
 ['。', '。', '。', '記号-句点', '', ''],
 ['自分自身', 'ジブンジシン', '自分自身', '名詞-固有名詞-一般', '', ''],
 ['の', 'ノ', 'の', '助詞-連体化', '', ''],
 ['若さ故の過ち', 'ワカサユエノアヤマチ', '若さ故の過ち', '名詞-固有名詞-一般', '', ''],
 ['という', 'トイウ', 'という', '助詞-格助詞-連語', '', ''],
 ['もの', 'モノ', 'もの', '名詞-非自立-一般', '', ''],
 ['を', 'ヲ', 'を', '助詞-格助詞-一般', '', ''],
 ['。', '。', '。', '記号-句点', '', ''],
 ['EOS'],
 ['']]

In [4]:
[line.split("\t") for line in mecab.parse("今日もがんばるぞい！").split("\n")]

[['今日', 'キョウ', '今日', '名詞-副詞可能', '', ''],
 ['も', 'モ', 'も', '助詞-係助詞', '', ''],
 ['がん', 'ガン', 'がん', '名詞-一般', '', ''],
 ['ばる', 'バル', 'バル', '名詞-一般', '', ''],
 ['ぞい', 'ゾイ', 'ぞい', '名詞-接尾-一般', '', ''],
 ['！', '！', '！', '記号-一般', '', ''],
 ['EOS'],
 ['']]

In [5]:
[line.split("\t") for line in mecab.parse("今日も一日がんばるぞい！").split("\n")]

[['今日も一日がんばるぞい', 'キョウモイチニチガンバルゾイ', '今日も一日がんばるぞい', '名詞-固有名詞-一般', '', ''],
 ['！', '！', '！', '記号-一般', '', ''],
 ['EOS'],
 ['']]

In [6]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]

In [7]:
def tokenizer(text, pos=False):
    text = re.sub(r"http://.*\.com", "", text)
    text = re.sub(r"【.*?】", "",text)
    text = text.lower()
    l = [line.split("\t") for line in mecab.parse(text).split("\n")]
    res = [i[0] if not pos else (i[0],i[3]) for i in l 
                   if len(i) >=4 
                       and ("名詞" in i[3] or "動詞" in i[3] or "形容詞" in i[3] )
                       and "数" not in i[3] and "助動詞" not in i[3] and "助詞" not in [3]
                       and  not re.search("[0-9]", i[0])
                       and i[0] not in stopwords
            ]
    return res

In [10]:
tokenizer("認めたくないものだな。自分自身の若さ故の過ちというものを。")

['認め', '自分自身', '若さ故の過ち']

In [11]:
tokenizer("認めたくないものだな。自分自身の若さ故の過ちというものを。", pos=True)

[('認め', '動詞-自立'), ('自分自身', '名詞-固有名詞-一般'), ('若さ故の過ち', '名詞-固有名詞-一般')]

In [12]:
doc_path = "./text/"
doc_dir = Path(doc_path)
dirs = [i for i in doc_dir.iterdir() if i.is_dir()]
dirs

[PosixPath('text/movie-enter'),
 PosixPath('text/it-life-hack'),
 PosixPath('text/kaden-channel'),
 PosixPath('text/topic-news'),
 PosixPath('text/livedoor-homme'),
 PosixPath('text/peachy'),
 PosixPath('text/sports-watch'),
 PosixPath('text/dokujo-tsushin'),
 PosixPath('text/smax')]

In [13]:
articles = [a for categ in dirs for a in categ.iterdir()]

In [14]:
num_docs = len(articles)
num_docs

7376

In [15]:
def read_doc(doc_id):
    with articles[doc_id].open() as f:
        print(f.read())

In [16]:
read_doc(5000)

http://news.livedoor.com/article/detail/5613985/
2011-06-08T08:00:00+0900
【Sports Watch】不振続く松井秀に「マイナーでのプレーを余儀なくされる」
日本時間6日のレッドソックス戦では、アスレチックス・松井秀喜に22打席ぶりのヒットが飛び出した。しかし、すでに自己ワーストの19打席無安打を超えるなど、打率は.213と低迷、苦しい日々が続いていることに変わりはない。

そんな折、7日発売の「週刊アサヒ芸能」（6.16号）では、在米のスポーツジャーナリストが同誌に対し、“背水”松井の周辺情報を伝えている。

「キャンプ時から近年になく、膝の状態はよかったのですが、松井のスロースターターぶりは例年どおりでした。夏場には打撃が復調することも見込まれ、慌てることもないでしょうが、5月に入ってチームが低迷するや、期待が高かっただけに他の選手に増して、DHに入る松井への風当たりが厳しくなってしまった」という在米ジャーナリスト。

今後の展望については、「地元紙記者は『ビリー（GM）がシーズン終了まで松井を置いておくことはありえない』と断言していました。トレード期限の7月30日までに、松井で商売するためにトレードを画策するでしょうね。でも、今の成績では高年俸の松井をプレーオフの切り札として獲得する球団も見当たらない。『日本ではやらない』と語っていた松井ですから、このままでは元レッドソックス・岡島秀樹(35)同様にマイナーでのプレーを余儀なくされるでしょう」と述べている。

・週刊アサヒ芸能 ［ライト版］＜デジタル＞（PC版）



In [17]:
def get_bow(doc_id):
    with articles[doc_id].open() as f:
        f.readline()
        f.readline()
        return tokenizer(f.read())

In [18]:
get_bow(5000)[:10]

['不振', '続く', '松井秀', 'マイナー', 'プレー', '余儀なく', 'さ', 'れる', '日本時間', 'レッドソックス']

In [19]:
docs = []
for a in tqdm.tqdm(articles):
    with a.open() as f:
        f.readline()
        f.readline()
        docs.append(tokenizer(f.read()))

100%|██████████| 7376/7376 [00:30<00:00, 240.17it/s]


In [20]:
docs[5000][:10]

['不振', '続く', '松井秀', 'マイナー', 'プレー', '余儀なく', 'さ', 'れる', '日本時間', 'レッドソックス']

In [21]:
d = corpora.Dictionary(docs)

stopwordsで排除しきれなかった邪魔な単語を処理

In [22]:
d.filter_extremes(no_below=5, no_above=0.2)
# nob_below以下の個数の単語を無視
# no_aboveの割合以上に出てくる単語を無視

d.compactify()
# idを振り直してコンパクトにする。

In [23]:
dic_num = len(d)
dic_num

24251

In [24]:
d[1000]

'長い'

In [25]:
d.doc2bow(docs[5000][:20])

[(475, 1),
 (531, 1),
 (1068, 1),
 (1391, 1),
 (3111, 1),
 (3954, 1),
 (4656, 1),
 (4999, 1),
 (7854, 1),
 (10364, 1),
 (13875, 1),
 (16596, 1),
 (18461, 1),
 (23591, 1),
 (23647, 1),
 (23687, 1),
 (23695, 1)]

(id, num in a doc)

In [26]:
d[1309] # get word from id

'若者'

In [27]:
corpus = [d.doc2bow(w) for w in docs]

In [28]:
corpus_words = sum(count for doc in corpus for id, count in doc)
corpus_words

1427891

In [29]:
num_topics = 10

In [30]:
logging.basicConfig(format='%(message)s', level=logging.INFO)

In [31]:
#lda = models.ldamodel.LdaModel(corpus=corpus, id2word=d, num_topics=num_topics, passes=10)
lda = models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, passes=10)

no word id mapping provided; initializing from corpus, assuming identity
using symmetric alpha at 0.1
using symmetric eta at 0.1
using serial LDA version on this node
running online (multi-pass) LDA training, 10 topics, 10 passes over the supplied corpus of 7376 documents, updating model once every 2000 documents, evaluating perplexity every 7376 documents, iterating 50x with a convergence threshold of 0.001000
PROGRESS: pass 0, at document #2000/7376
merging changes from 2000 documents into a model of 7376 documents
topic #2 (0.100): 0.008*"84" + 0.006*"504" + 0.005*"217" + 0.004*"89" + 0.004*"403" + 0.004*"14918" + 0.003*"5171" + 0.002*"5906" + 0.002*"1401" + 0.002*"24"
topic #7 (0.100): 0.009*"84" + 0.005*"5906" + 0.005*"3419" + 0.005*"1717" + 0.005*"403" + 0.004*"1535" + 0.003*"217" + 0.003*"89" + 0.003*"2425" + 0.003*"1748"
topic #8 (0.100): 0.009*"84" + 0.007*"217" + 0.004*"238" + 0.003*"403" + 0.003*"7774" + 0.003*"24" + 0.003*"7203" + 0.003*"5906" + 0.002*"1124" + 0.002*"122"
t

topic diff=0.640873, rho=0.419296
PROGRESS: pass 2, at document #2000/7376
merging changes from 2000 documents into a model of 7376 documents
topic #4 (0.100): 0.009*"1816" + 0.008*"2399" + 0.006*"707" + 0.005*"107" + 0.005*"435" + 0.005*"266" + 0.004*"267" + 0.004*"8" + 0.003*"2614" + 0.003*"1441"
topic #7 (0.100): 0.016*"1535" + 0.015*"3419" + 0.009*"466" + 0.009*"1748" + 0.007*"1717" + 0.006*"403" + 0.006*"986" + 0.005*"1514" + 0.005*"1727" + 0.005*"2425"
topic #0 (0.100): 0.009*"1832" + 0.007*"6106" + 0.006*"2105" + 0.006*"1514" + 0.005*"2802" + 0.005*"8253" + 0.005*"1729" + 0.005*"3386" + 0.004*"7774" + 0.004*"3850"
topic #3 (0.100): 0.032*"343" + 0.021*"156" + 0.015*"1104" + 0.012*"564" + 0.008*"1740" + 0.007*"3386" + 0.007*"2802" + 0.007*"8253" + 0.007*"7857" + 0.006*"3332"
topic #9 (0.100): 0.022*"84" + 0.012*"217" + 0.007*"504" + 0.007*"89" + 0.007*"403" + 0.004*"263" + 0.004*"99" + 0.004*"24" + 0.004*"238" + 0.003*"1401"
topic diff=0.607375, rho=0.386680
PROGRESS: pass 2, at 

topic #0 (0.100): 0.011*"1514" + 0.008*"1729" + 0.008*"1832" + 0.007*"3386" + 0.007*"8253" + 0.007*"7774" + 0.006*"2802" + 0.006*"3850" + 0.005*"9694" + 0.005*"1772"
topic #8 (0.100): 0.008*"2821" + 0.007*"729" + 0.005*"1124" + 0.005*"663" + 0.005*"4679" + 0.005*"181" + 0.005*"956" + 0.004*"1803" + 0.004*"4355" + 0.004*"662"
topic #9 (0.100): 0.022*"84" + 0.012*"217" + 0.008*"403" + 0.007*"504" + 0.007*"89" + 0.004*"263" + 0.004*"24" + 0.004*"99" + 0.004*"238" + 0.003*"1401"
topic diff=0.367466, rho=0.339266
PROGRESS: pass 4, at document #4000/7376
merging changes from 2000 documents into a model of 7376 documents
topic #2 (0.100): 0.014*"21582" + 0.006*"1687" + 0.006*"14918" + 0.005*"5171" + 0.003*"784" + 0.003*"869" + 0.002*"138" + 0.002*"231" + 0.002*"14228" + 0.002*"2614"
topic #6 (0.100): 0.005*"398" + 0.004*"634" + 0.004*"1724" + 0.003*"3165" + 0.003*"1959" + 0.003*"5574" + 0.003*"157" + 0.003*"2614" + 0.003*"1531" + 0.003*"1857"
topic #0 (0.100): 0.009*"1514" + 0.007*"1832" + 0.

topic diff=0.284119, rho=0.305880
PROGRESS: pass 6, at document #4000/7376
merging changes from 2000 documents into a model of 7376 documents
topic #5 (0.100): 0.010*"9227" + 0.007*"12751" + 0.005*"5176" + 0.005*"3850" + 0.005*"1055" + 0.005*"2090" + 0.005*"1609" + 0.005*"1412" + 0.004*"3153" + 0.004*"6989"
topic #4 (0.100): 0.009*"707" + 0.008*"2399" + 0.008*"1816" + 0.006*"266" + 0.005*"107" + 0.005*"435" + 0.005*"8" + 0.004*"1441" + 0.004*"970" + 0.004*"344"
topic #8 (0.100): 0.010*"729" + 0.009*"2821" + 0.009*"20819" + 0.007*"1124" + 0.006*"3829" + 0.006*"9972" + 0.005*"956" + 0.005*"663" + 0.005*"4355" + 0.005*"1803"
topic #6 (0.100): 0.005*"398" + 0.004*"634" + 0.004*"1724" + 0.004*"15349" + 0.003*"3165" + 0.003*"1959" + 0.003*"5574" + 0.003*"2614" + 0.003*"157" + 0.003*"339"
topic #2 (0.100): 0.016*"21582" + 0.007*"1687" + 0.006*"14918" + 0.005*"5171" + 0.003*"784" + 0.003*"138" + 0.003*"231" + 0.003*"2276" + 0.002*"14228" + 0.002*"1070"
topic diff=0.197627, rho=0.305880
PROGRES

topic #3 (0.100): 0.042*"343" + 0.027*"156" + 0.020*"564" + 0.018*"1104" + 0.010*"1740" + 0.010*"1069" + 0.010*"2802" + 0.009*"1545" + 0.009*"1066" + 0.009*"3332"
topic #1 (0.100): 0.014*"7782" + 0.013*"7203" + 0.013*"2105" + 0.012*"10647" + 0.012*"2562" + 0.011*"6106" + 0.009*"5906" + 0.008*"14584" + 0.008*"15348" + 0.008*"1104"
topic #0 (0.100): 0.012*"1514" + 0.008*"3386" + 0.008*"8253" + 0.007*"1729" + 0.007*"3850" + 0.007*"1717" + 0.007*"5906" + 0.007*"7774" + 0.006*"2802" + 0.006*"1832"
topic #8 (0.100): 0.010*"729" + 0.009*"2821" + 0.008*"20819" + 0.007*"1124" + 0.006*"3829" + 0.006*"9972" + 0.005*"956" + 0.005*"663" + 0.005*"1803" + 0.005*"4355"
topic diff=0.169054, rho=0.280739
PROGRESS: pass 8, at document #6000/7376
merging changes from 2000 documents into a model of 7376 documents
topic #6 (0.100): 0.007*"634" + 0.005*"398" + 0.004*"2614" + 0.004*"1724" + 0.003*"1959" + 0.003*"4122" + 0.003*"339" + 0.003*"15349" + 0.003*"5574" + 0.003*"157"
topic #1 (0.100): 0.014*"7782" + 

In [32]:
sorted(lda.get_document_topics(corpus[5000]), key=lambda t:t[1], reverse=True)

[(8, 0.92609888), (0, 0.028956642), (6, 0.023977976), (3, 0.015024432)]

In [33]:
def get_topic_words(topic_id):
    for t in lda.get_topic_terms(topic_id):
        print("{}: {}".format(d[t[0]], t[1]))

In [34]:
for t in range(num_topics):
    print("Topic # ",t)
    get_topic_words(t)
    print("\n")

Topic #  0
アプリ: 0.016738247126340866
ゲーム: 0.009150682017207146
利用: 0.009103640913963318
サービス: 0.008704044856131077
twitter: 0.007129988167434931
iphone: 0.007018717937171459
機能: 0.006462872494012117
ユーザー: 0.0062570772133767605
無料: 0.006255350075662136
情報: 0.0062307692132890224


Topic #  1
s: 0.01551967766135931
スマートフォン: 0.014645619317889214
max: 0.01438598521053791
d: 0.013437158428132534
対応: 0.012837924063205719
搭載: 0.010836421512067318
エスマックス: 0.010153787210583687
ソフトウェア: 0.009066270664334297
機能: 0.008114876225590706
更新: 0.007523820269852877


Topic #  2
独女: 0.028738105669617653
自転車: 0.01107153668999672
汗: 0.00418183533474803
google: 0.003195810131728649
マンガ: 0.0031624161638319492
イラスト: 0.0031144011300057173
コスメ: 0.003111886791884899
ケア: 0.0029995152726769447
対策: 0.002819277346134186
原因: 0.0027014955412596464


Topic #  3
ご: 0.044323429465293884
更新: 0.036675456911325455
ください: 0.031216522678732872
キャンペーン: 0.019126228988170624
実施: 0.012502143159508705
ダウンロード: 0.011652039363980293
情報: 

## perplexity

In [35]:
lda.log_perplexity(corpus)

-8.420 per-word bound, 342.4 perplexity estimate based on a held-out corpus of 7376 documents with 1427891 words


-8.4196976913145107

Estimate the variational bound of documents from corpus: E_q[log p(corpus)] - E_q[log q(corpus)]

In [36]:
bound = lda.bound(corpus)/corpus_words
bound #= lda.log_perplexity(corpus)

-8.4196999589439123

In [37]:
perplexity = np.exp2(-bound ) # 2^(-bound per words)

## see on tensorboard

http://projector.tensorflow.org/  
ref: https://github.com/RaRe-Technologies/gensim/blob/develop/docs/notebooks/Tensorboard_visualizations.ipynb


In [38]:
all_topics = lda.get_document_topics(corpus, minimum_probability=0)

In [39]:
with open('doc_lda_tensor.tsv','w') as w:
    for doc_topics in all_topics:
        for topics in doc_topics:
            w.write(str(topics[1])+ "\t")
        w.write("\n")    

In [40]:
meta = [str(a).split("/") for a in articles]

In [41]:
meta[0]

['text', 'movie-enter', 'movie-enter-5978741.txt']

In [42]:
with open('doc_lda_metadata.tsv','w') as w:
    w.write('Titles\tGenres\n')
    for m in meta:
        w.write("%s\t%s\n" % (m[1][:2], m[1]))

## 類似度

In [ ]:
doc_topics = np.array([lda[c] for c in corpus])

In [ ]:
doc_topics.shape

In [ ]:
dense = np.zeros( (len(doc_topics), num_topics), float)

for doc_id, t in enumerate(doc_topics):
    for topic_id, weight in t:
        dense[doc_id, int(topic_id)] = weight

例えば
$$
\text{doc_id(7376)}
\underbrace{
\begin{pmatrix}
0 & 0 & weight & \cdots & 0\\
0 & 0 & 0 & \cdots & 0\\
0 & weight & 0 & \cdots & 0\\
\vdots & \vdots & \vdots & \vdots & \vdots\\
0 & 0 & 0 & \cdots & 0
\end{pmatrix}
}_{\text{topics(20)}}
$$

In [ ]:
from scipy.spatial import distance
pairwise = distance.squareform(distance.pdist(dense, metric="cosine"))

$$
1 - \frac{u \cdot v}
         {{||u||}_2 {||v||}_2}
$$

In [ ]:
def similar_to(doc_id, num=5, distance=False):
    doc = pairwise[doc_id]
    ids = np.argsort(doc)[1: num+1] # the most similar is self.
    if distance:
        dist = np.sort(doc)[1: num+1]
        return [(x,y) for x, y in zip(ids, dist)]
    else:
        return ids

In [ ]:
read_doc(1000)

In [ ]:
similar_to(1000, num=10, distance=True)

In [ ]:
read_doc(988)

In [ ]:
read_doc(2580)

In [ ]:
from gensim import similarities

In [ ]:
doc_index = similarities.docsim.MatrixSimilarity(lda[corpus])

In [ ]:
c = corpus[5000]
vec_lda = lda[c]

In [ ]:
s = doc_index.get_similarities(vec_lda)

In [ ]:
s = sorted(enumerate(s), key=lambda t: t[1], reverse=True)

In [ ]:
s[1:11]

In [ ]:
similar_to(5000, num=10, distance=True)